# Deep Linking: Adding deep links and persistent sessions

Reference: https://panel.holoviz.org/how_to/state/index.html

## Basics

You can store a widget's current value by syncing it to the browser URL:

```python
import panel as pn

slider = pn.widgets.FloatSlider(name='Slider', start=0, end=10)

if pn.state.location:
    pn.state.location.sync(slider, {'value': 'slider_value'})  # Rename 'value' to 'slider_value' in the URL

slider.servable()
```

### Exercise 1

[Click here](/panel-preview/render/exercise_1.py) to try it out

1. Observe the browser's URL changing as you adjust the slider.
2. Experiment with setting a custom `slider_value` in the URL. Upon entry, the slider widget should match this custom value.
3. Copy this URL to a new browser tab. Once again, the slider widget's value should match the custom value.

![Exercise 1 Demo](assets/exercise_1.gif)

### Takeaway

You can persist a widget's state through the browser URL, making it easy to share interesting findings with colleagues!

## Accessing Session Args

You can pass parameters to the URL, without the need for widgets, and collect them using `session_args`:

```python
import panel as pn

set_price = 88

alert = pn.pane.Alert(alert_type="warning", visible=False)
item_image = pn.pane.Image("https://panel.holoviz.org/_static/logo.png")
price_shown = pn.pane.Markdown(object=f"## Price: ${set_price}")
layout = pn.Column(alert, item_image, price_shown)

promo_code = pn.state.session_args.get('promo_code', [b""])[0].decode()
if promo_code and promo_code.endswith("OFF"):
    discount = float(promo_code.replace("OFF", "")) / 100
    new_price = set_price * discount
    price_shown.object = f"## Price: $~~{set_price}~~ <font color='red'>Now ${new_price}</font>"
elif promo_code:
    alert.visible = True
    alert.object = f"The promo code provided, {promo_code!r}, is invalid."

layout.servable()
```

### Exercise 2

[Click here](/panel-preview/render/exercise_2.py?promo_code=50OFF) to try it out with a promo code!

1. Observe that, even without a promo code widget, you can pass a promo code as a parameter in the URL.
2. Experiment with changing the numeric part of the `promo_code`, for example, from `50OFF` to `10OFF` or even `100OFF`, and see how it affects the displayed price.
3. Try setting an invalid `promo_code`, like keeping only the numbers and excluding `OFF`. The alert pane should notify you that the promo code is invalid.
4. Finally, remove the `?promo_code` parameter to see that the app still loads!

![Exercise 2 Demo](assets/exercise_2.gif)

### Takeaway

Session parameters can be passed separately, independent of widgets. These session parameters can then be used to change the behavior of the app.

## Utilizing Parameterized Objects

Thus far, we only showed syncing with widgets, but it is also possible to synchronize them with `Parameterized` objects.

```python
import datetime

import param
import panel as pn
import pandas as pd
import hvplot.pandas

pn.extension()

URL = (
    "https://mesonet.agron.iastate.edu/cgi-bin/request/daily.py?network=DE__ASOS&stations={station}&"
    "year1={yesterday.year}&month1={yesterday.month}&day1={yesterday.day}&"
    "year2={today.year}&month2={today.month}&day2={today.day}"
    "&var=max_temp_f&na=blank&format=csv"
)

import datetime

import param
import panel as pn
import pandas as pd
import hvplot.pandas

pn.extension()

URL = (
    "https://mesonet.agron.iastate.edu/cgi-bin/request/daily.py?network=DE__ASOS&stations={station}&"
    "year1={yesterday.year}&month1={yesterday.month}&day1={yesterday.day}&"
    "year2={today.year}&month2={today.month}&day2={today.day}"
    "&var=max_temp_f&na=blank&format=csv"
)

class WeatherApp(param.Parameterized):
    station = param.Selector(default="EDDB", objects=["EDDB", "EDDM", "EDDH"])
    days = param.Integer(default=7, bounds=(1, 14))
    today = param.Date(default=datetime.date.today(), readonly=True)

    def __init__(self, **params):
        super().__init__(**params)
        pn.state.location.sync(self, {"station": "station", "days": "days"})

    @param.depends("station", "days")
    def _display_data(self):
        yesterday = self.today - datetime.timedelta(days=self.days)
        url = URL.format(station=self.station, today=self.today, yesterday=yesterday)
        df = pd.read_csv(url, index_col="day")
        return pn.Tabs(("Plot", df.hvplot(title=self.station)), ("Table", df))

    def panel(self):
        return pn.Row(
            # throttle
            pn.Param(
                self.param,
                parameters=["station", "days"],
                widgets={"days": {"throttled": True}},
            ),
            self._display_data,
        )


weather_app = WeatherApp()
weather_app.panel().servable()
```

### Exercise 3

[Click here](/panel-preview/render/exercise_3.py?station=EDDM&days=7) to test the app!

1. Observe that even though the default station is set to "EDDB," when the URL parameter is set to "EDDM," the app initially displays data for "EDDM."
2. Experiment with selecting different stations and numbers of days through both the widgets and the URL. You should notice that the widgets and URL parameters update accordingly.

![Exercise 3 Demo](assets/exercise_3.gif)

### Takeaway

Whether you choose a functional or class-based approach to create your apps, the URL and widgets stay in sync!

## Disabling Parameter Synchronization

At times, you may wish to stop synchronizing a widget or a `param.Parameterized` class when it becomes irrelevant to your application.

```python
import param
import panel as pn


class RoomPage(param.Parameterized):
    fan_speed = param.Number(default=1, bounds=(0, 1))
    lights_on = param.Boolean(default=False)

    def panel(self):
        return pn.Param(self, show_name=False)

    def _sync(self):
        pn.state.location.sync(self)

    def _unsync(self):
        pn.state.location.unsync(self)


class ControlCenter(param.Parameterized):
    room = param.Selector(objects=["Engine Room", "Office Room"])

    def __init__(self, **params):
        super().__init__(**params)
        self._room_pages = {
            "Engine Room": RoomPage(fan_speed=1, lights_on=False),
            "Office Room": RoomPage(fan_speed=0.5, lights_on=True),
        }
        self._last_page = None

        sidebar = pn.Column(self.param.room)
        self.main = pn.Column()
        self.template = pn.template.FastListTemplate(
            title="Control Center",
            sidebar=[sidebar],
            main=[self.main],
        )

        self.param.trigger("room")

    @param.depends("room", watch=True)
    def _update_page(self):
        if self._last_page is not None:
            self._last_page._unsync()

        room_page = self._room_pages[self.room]
        room_page._sync()
        self.main.objects = [room_page.panel()]

        self._last_page = room_page

    def panel(self):
        return self.template


control_center = ControlCenter()
control_center.panel().servable()
```

### Exercise 4

[Click here](/panel-preview/render/exercise_4.py) to try!

1. Initially, change the fan speed of the Engine Room to 0.8, and observe how the URL synchronizes with the widget values.
2. Select the Office Room; again, check out how the URL reflects the current room's widget values.
3. Next, choose a unique fan speed value, like 0.2.
4. Switch back to the Engine Room, and you'll see that both the widget and URL return to the previous value of 0.8 with lights off!

![Exercise 4 Demo](assets/exercise_4.gif)

## Stock Explorer

Now, it's your turn to implement location synchronization.

Imagine a scenario where you work as a stocks analyst, and you've discovered something remarkable involving two stock tickers, IBM and APPL. You want to share this insight with your colleague. However, currently, the app's state isn't shareable; the widgets reset when you share the URL.

Your task is to update `exercise_5.py` so that the location is synchronized!

[Click here to check if your modifications work!](/panel-preview/render/exercise_5.py)

For an additional challenge, picture a scenario where you manage a trading platform. The platform's users have their portfolios, which consist of lists of stock tickers. Your challenge is to display only the tickers that a user owns, based on `pn.state.user`.

## Conclusion


URL Parameter Synchronization in Panel offers a wide range of possibilities for creating interactive and shareable web applications. Here are some creative ideas for how you can leverage this feature:

Data Exploration Dashboards: Create a data exploration dashboard where users can adjust parameters such as time range, data filters, and visualization options through URL parameters. This allows users to share specific views of datasets with colleagues.

Custom Reports: Build a reporting tool that generates custom reports based on user-selected parameters. Users can save their report configurations as bookmarks by simply sharing the URL.

Dashboard Templates: Develop a template for common business metrics or KPI dashboards. Users can clone the template and customize it by adjusting URL parameters to match their specific data sources.

Educational Apps: Create interactive educational apps where teachers can set up exercises and share them with students through unique URLs. Students can work on assignments and save their progress via the URL.

A/B Testing: Implement A/B testing for website changes by using URL parameters to switch between different versions of a web page. This makes it easy to compare user engagement and conversion rates.

Interactive Maps: Build interactive maps that display different layers of geographic data. Users can toggle between layers and zoom levels through URL parameters, allowing for location-based data exploration.

Financial Portfolios: Develop a portfolio management tool where users can track their investments. Each portfolio can have its own unique URL, making it easy for users to access their financial data.

Language Learning: Create language learning exercises that adjust to the user's language proficiency level. Users can bookmark their progress and continue language lessons by revisiting the URL.

Real Estate Listings: Build a real estate listings website where users can filter properties by various criteria. Users can share their filtered property lists with friends and family via URL.

Game Configurations: Design browser-based games with adjustable game parameters like difficulty, starting resources, or character attributes. Gamers can share their favorite game setups with others.

Event Planning: Develop an event planning tool that allows users to customize event details such as date, location, and guest list. Event organizers can send out invitations with a unique URL for RSVPs.

Scientific Simulations: Create scientific simulations where researchers can adjust simulation parameters and share specific experimental setups with collaborators or the broader research community.

Stock Market Analysis: Build a stock market analysis tool where traders can set their preferred stock watchlist and technical indicators. Traders can share their analysis setups with peers.

Custom Surveys: Develop a survey tool where users can design custom surveys with various question types. Users can generate unique URLs for each survey and share them with respondents.

Healthcare Dashboards: Create healthcare dashboards where medical professionals can customize patient data views and share relevant patient information with colleagues.